In [ ]:
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import h5py
import numpy as np
import yaml
from quantum_annealing.src.energy_function import EnergyFunction

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Arial",
    "font.size": 20,
})

script_dir = os.getcwd()
# ---------- here is the path to the results directories ----------------------
mqlib_results_dir_path = f"{script_dir}/outputs/qbp/maxcut/2024-04-28/10-22-12"
annealing_results_dirs = f"{script_dir}/outputs/qbp/maxcut/"
# ---------------------------------------------------------------------------
simcim_results_path = f"{mqlib_results_dir_path}/simcim_result"
mqlib_results_path = f"{mqlib_results_dir_path}/mqlib_result"
results_path = f"{mqlib_results_dir_path}/result"
annealing_results_dirs_list = os.walk(annealing_results_dirs).__next__()[1]
with h5py.File(results_path, "r") as f:
    fields = np.array(f["fields"])
    nodes_number = fields.shape[0]
    vidal_distances_after_regauging = np.array(f["vidal_distances_after_regauging"])

total_evolution_times = []
annealing_energies = []
for annealing_results_dir in annealing_results_dirs_list:
    annealing_results_dir = f"{annealing_results_dirs}{annealing_results_dir}"
    subresults_dirs = os.walk(annealing_results_dir).__next__()[1]
    for subresults_dir in subresults_dirs:
        final_dir = f"{annealing_results_dir}/{subresults_dir}/result"
        config_path = f"{annealing_results_dir}/{subresults_dir}/.hydra/config.yaml"
        with open(config_path, "r") as f:
            raw_config = f.read()
            config = yaml.safe_load(raw_config)
            time_steps_num =int(config["quantum_annealing_schedule"]["steps_number"])
            time_step_size = float(config["quantum_annealing_schedule"]["total_time_step_size"])
            total_evolution_times.append(time_steps_num * time_step_size)
        with h5py.File(final_dir, "r") as f:
            annealing_energies.append(np.array(f["quantum_annealer_energy"]))
            pairs = np.array(f["coupled_spin_pairs"])
            truncation_errors = np.array(f["truncation_errors"])
            edges_number = pairs.shape[0]
            entropies = np.array(f["entropies"])
            entropies = entropies.reshape((-1, edges_number))
            fields = np.array(f["fields"])
            coupling_amplitudes = np.array(f["coupling_amplitudes"])
            energy_function = EnergyFunction(coupling_amplitudes, pairs, fields)
            sampled_config = np.array(f["quantum_annealer_configuration"])
time_indices = np.argsort(total_evolution_times)
total_evolution_times = np.array(total_evolution_times)[time_indices]
annealing_energies = np.array(annealing_energies)[time_indices]
with h5py.File(simcim_results_path, "r") as f:
    simcim_energy = np.array(f["simcim_energy"])
heuristics_energy = [simcim_energy]
heuristics_names = ["SIMCIM"]
with h5py.File(mqlib_results_path, "r") as f:
    for key in f.keys():
        if "energy" in key:
            energy = np.array(f[key])
            heuristics_energy.append(energy)
            heuristics_names.append(key.split("_")[0])
name_and_energy = list(zip(heuristics_names, heuristics_energy))
name_and_energy.sort(key=lambda x: x[1])
min_heuristic_energy = name_and_energy[0][1]
fig, ax = plt.subplots(1, 1)
sorted_cuts = list(map(lambda x: 0.5 * (-x[1] + edges_number), name_and_energy))
ax.scatter(
    np.arange(len(name_and_energy)),
    sorted_cuts,
    color="b",
)
ax.text(0, 0.5 * (-annealing_energies[0] + edges_number) - 1.8, r"GTQA with quantum measurements", fontdict={"fontsize" : 15})
ax.set_ylabel(r"Cut value", fontdict={"fontsize" : 18})
ax.set_ylim(top = 207, bottom = 170)
for i in np.arange(len(name_and_energy)):
    ax.plot([i, i], [0, sorted_cuts[i]], color="blue", linestyle='dotted', linewidth=0.7, alpha = 0.6)
axins = inset_axes(ax, width="100%", height="100%",
                   bbox_to_anchor=(.2, .2, .6, .6),
                   bbox_transform=ax.transAxes, loc=2, borderpad=0)
axins.plot(vidal_distances_after_regauging[-nodes_number:], 'b')
axins.set_ylabel(r"Vidal distance", fontsize=16)
axins.set_xlabel(r"Measured qubits fraction", fontsize=16)
axins.set_yscale("log")
axins.set_yticks([1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
axins.set_xticks([0, 75, 150])
axins.set_xticklabels([r"$0$", r"$0.5$", r"$1$"])
ax.set_xticks(np.arange(len(name_and_energy)), list(map(lambda x: x[0], name_and_energy)), rotation=90, family="monospace", fontsize=9)
ax.axhline(0.5 * (-annealing_energies[0] + edges_number), color='k', label=r"${\rm Cut: quantum \ annealing}$")
print(f"List of heuristics used: {list(map(lambda x: x[0], name_and_energy))}")
print(f"Total edges number: {edges_number}")
print(f"Best annealing cut: {int(0.5 * (-annealing_energies[0] + edges_number))}, best heuristic cut: {int(0.5 * (-min_heuristic_energy + edges_number))}")

In [4]:
fig.savefig(f"{mqlib_results_dir_path}/maxcut_benchmark.pdf",  bbox_inches="tight")